In [1]:
from pathlib import Path
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [2]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [3]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
1,EGRESO
2,NUEVO BENEFICIARIO
3,BENEFICIARIO RENOVANTE
4,DESERCION
5,GRATUIDAD


In [4]:
query_sum = """

SELECT id_2023 as id,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado

FROM cae
GROUP BY id, genero, colegio, region, quintil
"""

In [6]:
h = executes_query_duckdb(query_sum, cae)
h

,id,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado
0,461.230,M,Particular subvencionado,REGION DEL BIO-BIO,quintil 1 y 2,1,1,3,0,1,6366899.0
1,233.242,F,Estudiante de Colegio o Liceo Fiscal o Municipal,REGION DE LOS LAGOS,quintil 3,1,1,8,1,0,23154418.0
2,461.395,M,Municipal,REGION DE ÑUBLE,quintil 1 y 2,1,1,3,1,0,5497923.0
3,194.650,F,Particular subvencionado,REGION DEL LIB. B. O'HIGGINS,quintil 1 y 2,1,1,2,1,0,3692479.0
4,439.935,F,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,4,1,0,11329165.0
...,...,...,...,...,...,...,...,...,...,...,...
1262369,1.195.934,M,Municipal DAEM,REGION METROPOLITANA,quintil 3,1,1,1,0,0,5278206.0
1262370,1.207.590,M,Corporación Municipal,REGION METROPOLITANA,quintil 1 y 2,1,1,1,0,0,3720154.0
1262371,1.188.099,M,Municipal DAEM,REGION DE VALPARAISO,quintil 4,1,1,1,0,0,3729762.0
1262372,1.214.304,F,Particular Subvencionado,REGION DE ÑUBLE,quintil 1 y 2,1,1,1,0,0,3372000.0


In [37]:
cae.execute("""select * from cae limit 0""").df()

,id_2023,año_licitacion,año_operacion,genero_registrado,nacionalidad,region_grupo_familiar,quintil_ingreso_grupo_familiar,cambio_ies_carrera,tipo_beneficiario,nivel_estudio_al_ingreso,...,nombre_carrera,duracion_formal_carrera,contador_años_financiados,dependencia_colegio,arancel_solicitado,arancel_referencia,tipo_titulo,jornada,banco_administrador,total_prestado_ajustado
